# Resolution

In this lab, we will discuss resolution in the context of satellite imagery. We will focus on the four primary types:

1. Spatial
2. Temporal
3. Radiometric
4. Spectral

While doing this, we will introduce three of the most well-known satellite missions:
1. MODIS
2. Landsat
3. Sentinel

In [1]:
#!pip install geemap

import ee, geemap, pprint
#ee.Authenticate()

def build_map(lat, lon, zoom, vizParams, image, name):
    map = geemap.Map(center = [lat, lon], zoom = zoom)
    map.addLayer(image, vizParams, name)
    
    return map

# Initialize the Earth Engine module.
ee.Initialize()

## Spatial Resolution                                                     

Spatial resolution refers to the real-world representation of each pixel. This ranges widely, with the private satellite company Maxar announcing 15cm [resolution](https://blog.maxar.com/earth-intelligence/2020/introducing-15-cm-hd-the-highest-clarity-from-commercial-satellite-imagery), Sentinel at 10m, Landsat at 30m, and MODIS at 500. There are also large global products that have spatial resolution in the kilometers. The key point in dealing with spatial resolution is ensuring that your analysis drives your data collection, as there are tradeoffs involved. Using high resolution imagery can be expensive, both monetarily and computationally, if conducting continent wide analysis. Yet using low-resolution imagery will not be effective if your use case necessitates identifying individual buildings or small vehicles. Understanding the appropriate spatial resolution needed for your analysis is essential, and is why there are different platforms that focus on different spatial resolutions.

In practice, spatial resolution depends on the projection of the sensor's instantaneous field of view (IFOV) of the ground and how a set of radiometric measurements are resampled into a regular grid. To see the difference in spatial resolution resulting from different sensors, let's visualize data from the three primary platforms.

### MODIS 

There are two Moderate Resolution Imaging Spectro-Radiometers ([MODIS](http://modis.gsfc.nasa.gov/)) aboard the [Terra](http://terra.nasa.gov/) and [Aqua](http://aqua.nasa.gov/) satellites. Different MODIS [bands](http://modis.gsfc.nasa.gov/about/specifications.php) produce data at different spatial resolutions. For the visible bands, the lowest common resolution is 500 meters. Data from the MODIS platforms are used to produce a large number of data sets having daily, weekly, 16-day, monthly, and annual data sets. Outside this lab, you can find a list of MODIS land products [here](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table). 

In the code below, we are working with the MODIS Terra Surface Reflectance 8-day Global 500m resolution data. Change the number in the `zoom` variable to scroll in and out - notice that when scrolled in each pixel is quite large and granular. 

In [2]:
# Define the variables
lat = 13.7; lon = 2.54
zoom = 11
image_collection_name = 'MODIS/006/MOD09A1'
date_start = '2018-01-01'
date_end = '2018-05-01'
name = 'MODIS'

image = (
    ee.ImageCollection(image_collection_name)
         .filterDate(date_start, date_end)
         .first()
)

bands = ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03']

vizParams = {
    'bands': bands, 
    'min': -100,
    'max': 3000
}

map = build_map(lat, lon, zoom, vizParams, image, name)
map

Map(center=[13.7, 2.54], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

We will discuss some of the benefits of working with a false-color imagery in later sections, but we can modify the bands we want to visualize. In this case, we are using a random set of bands, where the value of band six is visualized with red, band three is visualized with green, and band one with blue. Because the value of band six has a higher range, this image shows up with a heavy red presence. 

In [3]:
bands = ['sur_refl_b06', 'sur_refl_b03', 'sur_refl_b01']

vizParams = {
    'bands': bands, 
    'min': -100, 
    'max': 3000
}

map1 = build_map(lat, lon, zoom, vizParams, image, name)
map1

Map(center=[13.7, 2.54], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

Compare the  size of MODIS pixels to objects on the ground. It may help to turn on the satellite basemap and lower the opacity of the layer (top right of map section of code editor) to see high-resolution data for comparison.

Print the size of the pixels (in meters) to the console. You can read more about how Google Earth Engine works with scale in their [documentation](https://developers.google.com/earth-engine/guides/scale). While the listed pixel resolution for this satellite platform is 500m, the printout is likely different - this is due to the way that GEE aggregates pixels to fit into a 256x256 tile. The details of this process are outside the scope of this course, but understand that GEE is conducting projections and resampling behind the scenes.  

### Landsat

Multi-spectral [scanners](https://landsat.gsfc.nasa.gov/multispectral-scanner-system) (MSS) were flown aboard Landsat missions 1-5 and have a spatial resolution of 60 meters. Let's look at the Landsat 5 MSS Collection 1 Tier 1 Raw Scenes - note that in the 'Bands' description of the dataset, there is no band related to blue (Green, Red, Near InfraRed 1 & 2). In this case, we will have to visualize the image using the near infrared, resulting in a false color composite. 

In [4]:
# Landsat 5
image_collection_name = 'LANDSAT/LT05/C02/T1_L2'
date_start = '1985-01-01'
date_end = '1989-12-31'
name = 'Landsat 5'


image = (
    ee.ImageCollection(image_collection_name)
         .filterDate(date_start, date_end)
         .median()
)

bands = ['SR_B3', 'SR_B2', 'SR_B1']

vizParams = {
    'bands': bands, 
    'min': 500, 
    'max': 25000
}

map2 = build_map(lat, lon, zoom, vizParams, image, name)
map2

Map(center=[13.7, 2.54], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

The Thematic Mapper ([TM](https://landsat.gsfc.nasa.gov/thematic-mapper/)) was flown aboard Landsat 4-5 and then succeeded by the Enhanced Thematic Mapper ([ETM+](https://landsat.gsfc.nasa.gov/the-enhanced-thematic-mapper-plus-etm/)) aboard Landsat 7 and the Operational Land Imager ([OLI](https://landsat.gsfc.nasa.gov/landsat-8/operational-land-imager)) / Thermal Infrared Sensor ([TIRS](https://landsat.gsfc.nasa.gov/landsat-8/thermal-infrared-sensor-tirs)) sensors aboard Landsat 8. TM data have a spatial resolution of 30 meters, which has remained the Landsat standard resolution. We can check this by importing the '*USGS Landsat 5 TM Collection 1 Tier 1 TOA Reflectance*', visualizing and printing our scale results. For some additional discussion about the transition from MSS to TM data, see [this page](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-5?qt-science_support_page_related_con=0#qt-science_support_page_related_con).

In [5]:
# Landsat 8
image_collection_name = 'LANDSAT/LC08/C02/T1_L2'
date_start = '2018-07-01'
date_end = '2018-12-01'
name = 'Landsat 8'

image = (
    ee.ImageCollection(image_collection_name)
        .filterDate(date_start, date_end)
        .median()
)

bands = ['SR_B4', 'SR_B3', 'SR_B2']

vizParams = {
    'bands': bands, 
    'min': 5000, 
    'max': 20000
}

map3 = build_map(lat, lon, zoom, vizParams, image, name)
map3

Map(center=[13.7, 2.54], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

### Sentinel
The Copernicus Program is a European incentive that is run by the European Space Agency (ESA). Sentinel is the satellite constellation that collects high-resolution and Synthetic Aperture Radar imagery globally. Sentinel has 10m resolution. 

In [6]:
image_collection_name = 'COPERNICUS/S2_SR'
date_start = '2019-01-01'
date_end = '2019-12-31'
name = 'Sentinel - Surface Reflection'

image = (
    ee.ImageCollection(image_collection_name)
         .filterBounds(ee.Geometry.Point(lon, lat))
         .filterDate(date_start, date_end)
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first()
)

bands = ['B4', 'B3', 'B2']

vizParams = {
    'bands': bands, 
    'min': 0, 
    'max': 3300
}

map4 = build_map(lat, lon, zoom, vizParams, image, name)
map4

Map(center=[13.7, 2.54], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

### High Resolution Data

Very high resolution data exists, but is not necessarily available. Companies such as Planet Labs and Maxar operate satellites that are cabable of collecting imagery in the sub-meter resolution range, and academics may be able to obtain sample data, but it is not generally available. 

The National Agriculture Imagery Program ([NAIP](http://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/)) is an effort by the USDA to acquire imagery over the continental US on a 3-year rotation using airborne sensors (aircraft as opposed to satellites). Because aircraft are much closer to land than a satellite (and is not dealing with as many atmospheric effects) imagery has a spatial resolution averaging 1 meter. This is considered 'high resolution data'.

Since NAIP imagery is distributed as 'quarters' of Digital Ortho Quads at irregular intervals, load everything from 2012 and [mosaic()](https://developers.google.com/earth-engine/guides/ic_composite_mosaic) the image together.

In [7]:

image_collection_name = "USDA/NAIP/DOQQ"
date_start = '2012-01-01'
date_end = '2012-12-31'
name = 'NAIP'
point = ee.Geometry.Point([-80.41, 37.23])

image = (
    ee.ImageCollection(image_collection_name)
            .filterDate(date_start, date_end)
            .filterBounds(point)
)

image = image.mosaic()

bands = ['R', 'G', 'B']

vizParams = {
    'bands': bands,
    #'min': 0, 
    #'max': 255
}

map5 = build_map(37.23, -80.41, 16, vizParams, image, name)
map5

Map(center=[37.23, -80.41], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

Look at the difference in the resolution - with Landsat and MODIS, each pixel could broadly identify the land type, but NAIP imagery has very high resolution - you can see individual parked cars, the outline of small trees, building envelopes, etc. Start asking yourself how the spatial resolutions of different platforms could help you answer unique questions.

Check the scale of NAIP by getting the first image from the mosaic (images within the mosaic might have different projections) and getting its scale (meters).

## Temporal Resolution

Temporal resolution refers to the *revisit time*, or how often the same satellite platform covers the same place on earth. Historically, satellites have been large, solitary objects that had to make tradeoffs between spatial and temporal resolution - MODIS measures wide swathes of land with each sweep, and has relatively high temporal resolution. Landsat has improved spatial resolution but a revisit rate of 16 days, and NAIP is aggregated either annually or bi-annually. Over the past decade, satellite technology has improved and there is more diversity in mission sets. Cube satellites are small, shoe-box sized satellites that can provide both high-resolution imagery and, when mosaiced together, provide high temporal resolution as well. The tradeoff is that these satellites do not have the same array of sophisticated sensors that larger satellites are equipped with. Other satellites, such as those run by the intelligence community and private satellite companies, are designed for rapid revisit times of certain cities or political areas while not scanning the rest of the world. 

Temporal resolution is important to understand and consider for your use case - there are tradeoffs to be made either way.

Resolution of a few popular platforms:

1. Landsat:    16 days
2. MODIS:      Several satellites, temporal resolution varies by product (4 days to annual products)
3. Sentinel:   5 days at equator
4. NAIP:       Annual
5. Planet:     Daily

## Spectral Resolution

Spectral resolution refers to the number and width of spectral bands in which the sensor takes measurements. You can think of the width of spectral bands as the wavelength interval on the electromagnetic spectrum for each band. A sensor that measures radiance in multiple bands (e.g., collects a value for red, green, blue and near infrared) is called a *multispectral* sensor (generally 3-10 bands), while a sensor with many bands (possibly hundreds) is called a *hyperspectral* sensor (these are not hard and fast definitions). For example, compare the [multi-spectral OLI](http://landsat.gsfc.nasa.gov/?p=5779) aboard Landsat 8 to [Hyperion](https://eo1.usgs.gov/sensors/hyperioncoverage), a hyperspectral sensor that collects 220 unique spectral channels aboard the EO-1 satellite.

You will have to read through the documentation for each image collection to understand the spectral response of the bands. 

![Specctral Ranges per Band](https://github.com/ghidora77/03_GEE_Labs_DSPG/blob/main/im/im_01_10.png?raw=true)

**Important note**: not all bands contain radiometric data. Some are quality control data, while others include information about the zenith or cloud coverage. You can use these other bands to either mask out low-quality pixels or conduct additional calculations. It is a good idea to read through the documentation of each dataset you will be working with to get a good understanding of the band structure.

## Radiometric Resolution

Radiometric resolution refers to the value, or 'digital number' that the sensor records: _coarse_ radiometric resolution would record a scene with only a narrow range of values, whereas _fine_ radiometric resolution would record the same scene using a wide range of values. The _precision_ of the sensing, or the level of _quantization_ is another way to refer to radiometric resolution. 8 bit values (0-255) is the standard in many image processing tools. 

![Radiometric Resolution](https://github.com/ghidora77/03_GEE_Labs_DSPG/blob/main/im/im_01_12.jpeg?raw=true)

Radiometric resolution is determined from the minimum radiance to which the detector is sensitive (L<sub>min</sub>), the maximum radiance at which the sensor saturates (L<sub>max</sub>), and the number of bits used to store the DNs (Q): 


$$  \text{Radiometric resolution} = \frac{(L_{max} - L_{min})}{2^Q} $$

It might be possible to dig around in the metadata to find values for L<sub>min</sub> and L<sub>max</sub>, but computing radiometric resolution is generally not necessary unless you're studying phenomena that are distinguished by very subtle changes in radiance. One thing to keep in mind is that while sensors have developed and become more sensitive / accurate, capable of recording collecting data in upwards of 16 bits, that may not necessarily be beneficial for your work. Computation and storage costs grow, and normalizing the data to 8-bit values to work with tools such as OpenCV defeats the purpose of this sensitive colllection rate. There are use cases where high bit rate collection makes sense (e.g., looking for a very narrow range in a custom spectral range to identify mineral deposits), but ensure that you understand where and why higher radiometric resolution is necessary. 

### Digital Image Visualization and Stretching

You've learned about how an image stores pixel data in each band as digital numbers (DNs) and how the pixels are organized spatially. When you add an image to the map, Earth Engine handles the spatial display for you by recognizing the projection and putting all the pixels in the right place. However, you must specify how to stretch the DNs to fit the standard 8-bit display image  that GEE uses ( `min` and `max` parameters). Specifying `min` and `max` applies (where DN' is the displayed value):

   $$ DN' =   \frac{ 255 (DN - min)}{(max - min)} $$

For instance, if you are working with NAIP imagery, you can set the min radiometric resolution to 0 and the max to 255 to model 8-bit radiometric resolution. 

By contrast, the Planet MultiSpectral SkySat imagery uses 16 bit collection, so you have to adjust the min and max values.  If your image is not displaying correctly (such as a black screen) check the documentation for your data and adjust your min and max values. 

In [8]:
lat = 41.66; lon = -70.9; 
zoom = 14
image_collection_name = 'SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL'
date_start = '2019-01-01'
date_end = '2019-12-31'
name = 'NAIP'
point = ee.Geometry.Point([lon, lat])

image = (
    ee.ImageCollection(image_collection_name)
         .median()
)

bands = ['N', 'G', 'B']

vizParams = {
    'bands': bands, 
    'min': 200, # Change to 200
    'max': 6000 # 6000
}

map6 = build_map(lat, lon, zoom, vizParams, image, name)
map6

Map(center=[41.66, -70.9], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…